In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_algs = pd.read_pickle('dataframes/df_all_algorithms.pkl')
df_mitdb = pd.read_pickle('dataframes/df_mitdb.pkl')

In [2]:
df_mitdb.head()

,ECG,True_peak,Peak_type
100,"[-0.145, -0.145, -0.145, -0.145, -0.145, -0.14...","[18, 77, 370, 662, 946, 1231, 1515, 1809, 2044...","[+, N, N, N, N, N, N, N, A, N, N, N, N, N, N, ..."
101,"[-0.345, -0.345, -0.345, -0.345, -0.345, -0.34...","[7, 83, 396, 711, 1032, 1368, 1712, 2036, 2349...","[+, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ..."
102,"[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....","[68, 136, 410, 697, 989, 1305, 1614, 1911, 220...","[+, /, /, /, /, /, /, /, /, /, /, /, /, /, /, ..."
103,"[-0.375, -0.375, -0.375, -0.375, -0.375, -0.37...","[21, 265, 575, 876, 1180, 1482, 1795, 2127, 24...","[+, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ..."
104,"[-0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0....","[36, 73, 316, 615, 901, 1188, 1477, 1749, 2062...","[+, /, /, /, /, /, f, f, /, /, /, /, /, f, f, ..."


In [3]:
df_algs.head()

,Alg1,Alg2,Alg3,Alg4,Alg5
100,"[79, 372, 665, 949, 1233, 1517, 1812, 2046, 24...","[663, 947, 1231, 1515, 1809, 2044, 2402, 2706,...","[72, 374, 666, 950, 1235, 1519, 1813, 2048, 24...","[111, 384, 693, 960, 1262, 1529, 1810, 2058, 2...","[305, 704, 987, 1273, 1556, 1850, 2087, 2438, ..."
101,"[85, 398, 713, 1034, 1370, 1714, 2038, 2351, 2...","[1032, 1368, 1711, 2035, 2348, 2661, 2971, 330...","[78, 400, 715, 1037, 1372, 1716, 2040, 2353, 2...","[118, 432, 727, 1067, 1384, 1747, 2051, 2384, ...","[333, 753, 1073, 1384, 1725, 2072, 2391, 2704,..."
102,"[88, 168, 413, 700, 991, 1308, 1616, 1913, 220...","[409, 696, 988, 1304, 1613, 1910, 2201, 2488, ...","[113, 428, 717, 1005, 1326, 1629, 1930, 2221, ...","[132, 441, 697, 1020, 1339, 1652, 1948, 2232, ...","[63, 453, 740, 1032, 1347, 1656, 1954, 2245, 2..."
103,"[82, 276, 578, 887, 1192, 1484, 1806, 2139, 24...","[575, 876, 1180, 1482, 1795, 2127, 2444, 2744,...","[79, 270, 580, 881, 1185, 1487, 1800, 2132, 24...","[129, 247, 590, 910, 1195, 1483, 1809, 2142, 2...","[201, 617, 918, 1222, 1524, 1835, 2168, 2485, ..."
104,"[19, 317, 616, 903, 1189, 1489, 1751, 2063, 23...","[613, 899, 1186, 1476, 1748, 2060, 2359, 2649,...","[20, 1481, 1752, 2111, 2409, 2653, 2992, 3239,...","[351, 648, 934, 1225, 1469, 1768, 2099, 2398, ...","[254, 657, 944, 1231, 1510, 1793, 2105, 2404, ..."


In [4]:
WINDOW = 50

def reference_algorithm(ref_algorithm, true_peaks, annotations):
    detections_annotations = []

    for detection in ref_algorithm:
        mask = (true_peaks > detection-WINDOW) & (true_peaks < detection+WINDOW)
        detected_is_true = true_peaks[mask]

        if len(detected_is_true) == 1:
            index = np.where(true_peaks == int(detected_is_true))
            annotation = annotations[index]
            detections_annotations.append(str(annotation[0]))
    
        elif len(detected_is_true) > 1:
            detections_annotations.append("AMB")
        
        else:
            detections_annotations.append("FP")

    return detections_annotations


def compute_differences(ref_algorithm, *algorithms):
    differences = []
    for peak in ref_algorithm:
        current_differences = []
        for alg in algorithms:
            mask = (alg > peak-WINDOW) & (alg < peak+WINDOW)
            other_detections = alg[mask]

            if len(other_detections) > 0:
                current_differences.append(int(other_detections[0]) - peak)
            else:
                current_differences.append(-WINDOW*2)

        differences.append(current_differences)
        
    return differences
    

In [5]:

alg1, alg2, alg3, alg4, alg5 = df_algs['Alg1'], df_algs['Alg2'], df_algs['Alg3'], df_algs['Alg4'], df_algs['Alg5']

true_peaks, annotations = df_mitdb['True_peak'], df_mitdb['Peak_type']

In [6]:
def make_differecnes_dataframe():
    df_diff = pd.DataFrame(columns=['alg1', 'alg3', 'alg4', 'alg5', 'annotations'])

    for case in df_mitdb.index:
        det_ann = reference_algorithm(alg2[case], true_peaks[case], annotations[case])
        diffs = compute_differences(alg2[case], alg1[case], alg3[case], alg4[case], alg5[case])

        diff = np.array(diffs)

        ann = np.array(det_ann).reshape(1, -1)

        data_to_append = np.hstack((diff, ann.T))

        # print(data_to_append)

        df_to_append = pd.DataFrame(data_to_append, columns=['alg1', 'alg3', 'alg4', 'alg5', 'annotations'])
        df_diff= pd.concat([df_diff, df_to_append], ignore_index=True)
        
    return df_diff 

In [7]:
df_diff = make_differecnes_dataframe()

In [8]:
df_diff.head()

,alg1,alg3,alg4,alg5,annotations
0,2,3,30,41,N
1,2,3,13,40,N
2,2,4,31,42,N
3,2,4,14,41,N
4,3,4,1,41,N


In [9]:
name = 'dataframes\df_input_to_classifier.pkl'
df_diff.to_pickle(name)
print(f"Dataframe saved as {name}")

Dataframe saved as dataframes\df_input_to_classifier.pkl
